# **Notebook EC04 (Part B)**
# MLP classifier for the MNIST database.
## Simplified grid search for the hyperparameters.
**Professor:** Fernando J. Von Zuben <br>
**Aluno(a):** Daniele Souza Gonçalves


In [7]:
import tensorflow as tf
import os
import pandas as pd
import matplotlib.pyplot as plt
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [8]:
def model_and_evaluate(N_NEURONS=512,N_LAYERS=1,DROPOUT=0.3,EPOCHS=5,
                       OPTIMIZER='adam',verbose=1):
    model = tf.keras.models.Sequential([
          tf.keras.layers.Flatten(),
          tf.keras.layers.Dense(N_NEURONS,activation=tf.nn.relu),
          tf.keras.layers.Dropout(DROPOUT)])
    if N_LAYERS>1:
        for i in range(N_LAYERS-1):
            model.add(tf.keras.layers.Dense(N_NEURONS,activation=tf.nn.relu))
            model.add(tf.keras.layers.Dropout(DROPOUT))
    model.add(tf.keras.layers.Dense(10, activation=tf.nn.softmax))
    model.compile(optimizer=OPTIMIZER,
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
    model_history = model.fit(x_train, y_train, epochs=EPOCHS,verbose=verbose)
    results = model.evaluate(x_test, y_test,verbose=verbose)
    if verbose:
        model.summary()
        # Evaluate the model on the test data using `evaluate`
        print("Evaluate on test data")
        print("test loss, test acc:", results)
    d = dict(n_neurons = N_NEURONS,
           n_layers  = N_LAYERS,
           optimizer = OPTIMIZER,
           dropout   = DROPOUT,
           epochs    = EPOCHS,
           loss      = results[0],
           accuracy  = results[1])
    return d

In [9]:
rows = []
for rep in range(10):
    row = model_and_evaluate(verbose=0)
    row['rep'] = rep
    rows.append(row)
    for n_neurons in [256,1024,2048,4096]:
        row = model_and_evaluate(N_NEURONS=n_neurons,verbose=0)
        row['rep'] = rep
        rows.append(row)
    for n_layers in [2,3,4]:
        row = model_and_evaluate(N_LAYERS=n_layers,verbose=0)
        row['rep'] = rep
        rows.append(row)
    for dropout in [0,0.1,0.2,0.4,0.5]:
        row = model_and_evaluate(DROPOUT=dropout,verbose=0)
        row['rep'] = rep
        rows.append(row)
    for optimizer in ['RMSprop','SGD','Ftrl']:
        row = model_and_evaluate(OPTIMIZER=optimizer,verbose=0)
        row['rep'] = rep
        rows.append(row)
    for epoch in [10,20,30]:
        row = model_and_evaluate(EPOCHS=epoch,verbose=0)
        row['rep'] = rep
        rows.append(row)
df = pd.DataFrame(rows)
df.to_csv('mnist_EC4_MLP_analysis.csv',index=False)
df.head()

2023-09-10 11:48:38.018981: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-09-10 11:49:21.148933: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-09-10 11:49:22.782179: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-09-10 11:50:03.754620: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-09-10 11:50:05.254068: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-09-10 11:50:53.087627: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-09-10 11:50:54.562864: I tensorflow/core/grappler/optimizers/cust

InvalidArgumentError: Cannot assign a device for operation sequential_32/dense_76/MatMul/ReadVariableOp: Could not satisfy explicit device specification '' because the node {{colocation_node sequential_32/dense_76/MatMul/ReadVariableOp}} was colocated with a group of nodes that required incompatible device '/job:localhost/replica:0/task:0/device:GPU:0'. All available devices [/job:localhost/replica:0/task:0/device:CPU:0, /job:localhost/replica:0/task:0/device:GPU:0]. 
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=2 requested_device_name_='/job:localhost/replica:0/task:0/device:GPU:0' assigned_device_name_='/job:localhost/replica:0/task:0/device:GPU:0' resource_device_name_='/job:localhost/replica:0/task:0/device:GPU:0' supported_device_types_=[CPU] possible_devices_=[]
ResourceApplyFtrl: CPU 
Mul: GPU CPU 
AddV2: GPU CPU 
RealDiv: GPU CPU 
ReadVariableOp: GPU CPU 
Const: GPU CPU 
_Arg: GPU CPU 

Colocatio

In [ ]:
df = pd.read_csv('mnist_EC4_MLP_analysis.csv')
df.head()

In [ ]:
# For the CIFAR10, if the outer loop proves to be too costly, use the code as follows
rows = []
for rep in [0]: # up to [for rep in [9]:]
    row = model_and_evaluate(verbose=0)
    row['rep'] = rep
    rows.append(row)
    for n_neurons in [256,1024,2048,4096]:
        row = model_and_evaluate(N_NEURONS=n_neurons,verbose=0)
        row['rep'] = rep
        rows.append(row)
    for n_layers in [2,3,4]:
        row = model_and_evaluate(N_LAYERS=n_layers,verbose=0)
        row['rep'] = rep
        rows.append(row)
    for dropout in [0,0.1,0.2,0.4,0.5]:
        row = model_and_evaluate(DROPOUT=dropout,verbose=0)
        row['rep'] = rep
        rows.append(row)
    for optimizer in ['RMSprop','SGD','Ftrl']:
        row = model_and_evaluate(OPTIMIZER=optimizer,verbose=0)
        row['rep'] = rep
        rows.append(row)
    for epoch in [10,20,30]:
        row = model_and_evaluate(EPOCHS=epoch,verbose=0)
        row['rep'] = rep
        rows.append(row)
df = pd.DataFrame(rows)
df.to_csv('cifar10_EC4_MLP_analysis.csv', mode='a', header=False, index=False)
df.head()

In [ ]:
n_neurons = [256,512,1024,2048,4096]
n_layers  = [1,2,3,4]
dropout   = [0,0.1,0.2,0.3,0.4,0.5]
optimizer = ['adam','RMSprop','SGD','Ftrl']
epoch     = [5,10,20,30]

accs = []
losss= []
for i,nn in enumerate(n_neurons):
    c = (df['n_neurons'] == nn) & (df['n_layers'] == n_layers[0])
    c = c & (df['dropout'] == dropout[3]) & (df['optimizer'] == optimizer[0])
    c = c & (df['epochs'] == epoch[0])
    acc = df[c]['accuracy']
    accs.append(acc)
    loss= df[c]['loss']
    losss.append(loss)

fig,ax=plt.subplots(2,1,figsize=(4,4),dpi=120,sharex=True)
ax[0].boxplot(accs,labels=n_neurons)
ax[0].set_ylabel('Accuracy')
ax[1].boxplot(losss,labels=n_neurons)
ax[1].set_ylabel('Loss')
ax[1].set_xlabel('Number of artificial neurons')
fig.align_ylabels()
plt.tight_layout()

In [ ]:
accs = []
losss= []
for i,nl in enumerate(n_layers):
    c = (df['n_neurons'] == 512) & (df['n_layers'] == nl)
    c = c & (df['dropout'] == dropout[3]) & (df['optimizer'] == optimizer[0])
    c = c & (df['epochs'] == epoch[0])
    acc = df[c]['accuracy']
    accs.append(acc)
    loss= df[c]['loss']
    losss.append(loss)

fig,ax=plt.subplots(2,1,figsize=(4,4),dpi=120,sharex=True)
ax[0].boxplot(accs,labels=n_layers)
ax[0].set_ylabel('Accuracy')
ax[1].boxplot(losss,labels=n_layers)
ax[1].set_ylabel('Loss')
ax[1].set_xlabel('Number of hidden layers')
fig.align_ylabels()
plt.tight_layout()

In [ ]:
accs = []
losss= []
for i,do in enumerate(dropout):
    c = (df['n_neurons'] == 512) & (df['n_layers'] == 1)
    c = c & (df['dropout'] == do) & (df['optimizer'] == optimizer[0])
    c = c & (df['epochs'] == epoch[0])
    acc = df[c]['accuracy']
    accs.append(acc)
    loss= df[c]['loss']
    losss.append(loss)

fig,ax=plt.subplots(2,1,figsize=(4,4),dpi=120,sharex=True)
ax[0].boxplot(accs,labels=dropout)
ax[0].set_ylabel('Accuracy')
ax[1].boxplot(losss,labels=dropout)
ax[1].set_ylabel('Loss')
ax[1].set_xlabel('Dropout')
fig.align_ylabels()
plt.tight_layout()

In [ ]:
accs = []
losss= []
for i,o in enumerate(optimizer):
    c = (df['n_neurons'] == 512) & (df['n_layers'] == 1)
    c = c & (df['dropout'] == 0.3) & (df['optimizer'] == o)
    c = c & (df['epochs'] == epoch[0])
    acc = df[c]['accuracy']
    accs.append(acc)
    loss= df[c]['loss']
    losss.append(loss)

fig,ax=plt.subplots(2,1,figsize=(4,4),dpi=120,sharex=True)
ax[0].boxplot(accs,labels=optimizer)
ax[0].set_ylabel('Accuracy')
ax[1].boxplot(losss,labels=optimizer)
ax[1].set_ylabel('Loss')
fig.align_ylabels()
plt.tight_layout()

In [ ]:
epoch     = [5,10,20,30]
accs = []
losss= []
for i,e in enumerate(epoch):
    c = (df['n_neurons'] == 512) & (df['n_layers'] == 1)
    c = c & (df['dropout'] == 0.3) & (df['optimizer'] == 'adam')
    c = c & (df['epochs'] == e)
    acc = df[c]['accuracy']
    accs.append(acc)
    loss= df[c]['loss']
    losss.append(loss)

fig,ax=plt.subplots(2,1,figsize=(4,4),dpi=120,sharex=True)
ax[0].boxplot(accs,labels=epoch,showmeans=True,meanline=True)
ax[0].set_ylabel('Accuracy')
ax[1].boxplot(losss,labels=epoch,showmeans=True,meanline=True)
ax[1].set_ylabel('Loss')
ax[1].set_xlabel('Epochs')
fig.align_ylabels()
plt.tight_layout()

In [ ]:
# Please, replace the values obtained by the simplified grid search here and execute the code
d = model_and_evaluate(N_NEURONS=???,N_LAYERS=???,DROPOUT=???,EPOCHS=???,
                       OPTIMIZER='???',verbose=1)